# Markov Chain Monte Carlo (MCMC) simulation in __INPUT_DATASET_SMART_NAME__

In the Bayesian methodology, the observations from a dataset are considered to be generated by a random process, which itself is parametrized by random variables. The goal of *Bayesian inference* is then to produce an estimate of the *posterior probability distribution* of those parameters.

In this notebook, we are going to select univariate observations from a dataset in the Flow, and leverage **Markov Chain Monte Carlo (MCMC)** in order to apply a Bayesian inference analysis on them. We leverage the capabilities of the pymc3 Python package, that allows to cleanly define the inference process and provides multiple sampling options.

Even though our use of Monte-Carlo methods (and Bayesian inference) here is mostly targeting a "generic" analysis, it can also be used within a broad range of specialized/targeted applications, such as survival analysis for churn prevention or unsupervised clustering for segmentation under uncertainty.

In [4]:
%matplotlib inline

import numpy as np
import math
import pymc3 as pm
import matplotlib.pyplot as plt
import scipy.stats
import warnings

warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'numpy.distutils'

## 1. Selecting the observation vector

The first task is to extract the observations of interest from our input dataset.

In [ ]:
# Choose the column containing the observations:
obs_column = "Customers"

In [ ]:
my_dataset = dataiku.Dataset("__INPUT_DATASET_SMART_NAME__")
dataset_columns = [x["name"] for x in my_dataset.read_schema()]
if obs_column not in dataset_columns:
    raise ValueError("Column {} does not exist".format(obs_column))
else:
    obs = my_dataset.get_dataframe(columns=[obs_column]).values

Now that we have our observation vector, we can take a look at its histogram to get a first overview of its distribution.

In [ ]:
fig = plt.figure(figsize=(8,7))
plt.title("{} - observed distribution".format(obs_column))
plt.xlabel(obs_column)
plt.hist(obs, bins=64);

## 2. Defining and running the probabilistic model

After assessing the distribution of the observations, you can now make an assumption about the nature of the random process that generated those observations. 

* In practice, we need to associate this process with a statistical distribution and its associated parameters: the probability of the observations given those parameters is called the *likelihood*. 

* In the Bayesian context, we consider those parameters to be random variables: the end goal is to obtain their *posterior probability distribution*, namely their probability conditioned by the observations. 

* To do so, we need to define the *prior distributions* of the parameters, which act as first-guess provided by the user about their statistical behaviour.

* We will then run a *sampler* which will iteratively:
    * draw values from the parameter priors,
    * compute their likelihood given the observations,
    * apply Bayes theorem to get a probability value for their posterior distribution,
    * optimize the sampler parameters to guide it towards areas of higher posterior probability.

###  Likelihood and prior

The first item to define is the likelihood function. The Gaussian distribution (`normal`) often fits well to describe observations, but there are also some cases where specific distributions are more appropriated, e.g. a Poisson distribution (`poisson`) for counts.

* You will have to define the likelihood parameters `lk_params` directly within the context manager (inside the `with` statement) before launching the sampler.

* If the type of likelihood you are looking for is not in the `lk_options` dictionary from the next cell, you can add a new entry by choosing from [all the different distributions available from pymc3](https://docs.pymc.io/api/distributions.html).

In [114]:
# Select the desired type of likelihood (must be part of lk_options dictionary!) 
lk_type = "normal"

In [ ]:
# Append additional likelihood shortcuts here:
lk_options = {"poisson": pm.Poisson,
              "normal": pm.Normal}

The following item to define is a prior distribution for each of the likelihood's parameters. The choice of priors is a vast topic that we simplify here by choosing a uniform distribution, which boundaries you can specify as being the most coherent with your use-case.

* If you are working with a Gaussian likelihood, you should also set up the `prior_sigma` parameter, which is the asserted value of the standard deviation. The simplest case is to assume a fixed value that reflects the potential spread of your observations around the inferred mean.

In [ ]:
# Put prior hyperparameters here:
unif_lower = 0
unif_upper = 2000

# For a Gaussian likelihood:
prior_sigma = 1000

###  Simulation parameters

The next step is to define the simulation parameters used by the sampler, as well as the sampler type.

A sampler is a given methodology for exploring the space of parameters and optimize that search for maximizing the posterior probability. In the case of MCMC, simply put, you progressively build a Markov chain that has the posterior distribution of the parameters as its equilibrium distribution.

There are several important parameters to set up for the sampler, the main ones are :
* the number of Markov chains to run simultaneously (`n_chains`): it is important to build at least 2 different chains that will validate the robustness of the sampling outcome by displaying similar results
* the number of draws per chain to be performed (`n_draws`): the sampler should do enough iterations for the Markov chains to be able to properly converge towards their stationary distribution. However, too many iterations may also lead to a slow computation, especially if the likelihood is costly to evaluate at each iteration.
* the algorithm used for sampling (`sampler_type`): there are numerous methods available to perform MCMC, the most common and generic ones in the pymc3 package are the *Metropolis-Hastings* (`metropolis`) and the *No-U-Turn sampler* (`nuts`).

If the type of sampler you are looking for is not in the `sampler_options` dictionary from the next cell, you can add a new entry by choosing from [all the different samplers available from pymc3](https://docs.pymc.io/api/inference.html#step-methods).

In [ ]:
sampler_options = {"metropolis": pm.Metropolis,
                   "nuts": pm.NUTS}

In [ ]:
n_chains = 3 # nb of independent Markov chains to be run in total
n_draws = 100000 # nb of samples to draw per Markov chain

sampler_type = "metropolis" 

The final step is then to create a *model* instanciated in the form of a context manager where you will define all the elements of your probabilistic model.

**Make sure to properly define the prior and `lk_params` according to your desired probabilistic model.**

Once you execute the next cell, the simuation will start running, and depending on the data and the simulation parameters, it can take a decent amount of time.

In [ ]:
with pm.Model() as my_model:
    # Define the parameter(s) to perform inference on:
    mu_lk = pm.Uniform('mu_lk', lower=unif_lower, upper=unif_upper)
    
    # Define the likelihood parameters:
    lk_params = {"mu": mu_lk, "sd": prior_sigma}
    # Additional keys to reference variable type and observations (do not change!)
    lk_params["name"] = "likelihood"
    lk_params["observed"] = obs
    # Instanciate actual likelihood distribution:
    likelihood = lk_options[lk_type](**lk_params)
    print("Chosen likelihood is {}".format(lk_type))
    
    # Initialize the sampler at the maximum a posteriori (MAP) for faster convergence:
    start = pm.find_MAP()
    
    # Enforce sampler type and options:
    step = sampler_options[sampler_type]()
    
    # Launch sampling and store output results in the trace variable:
    trace = pm.sample(draws=n_draws,
                      chains=n_chains,
                      start=start,
                      step=step)

## 3. Displaying and analyzing sampling results

### Summary statistics

A quick way of looking at the results of the sampling operation is to produce a small DataFrame that contains a set of summary statistics, some of them being especially useful to assess convergence:

- `Rhat` shoud be <1.05
- `n_eff` is the *effective sample size* and reflects the number of effective number of simulation draws that are truly independent: the higher, the better.

In [ ]:
pm.summary(trace, varnames=['mu_lk'])

### Trace frequency and values

A first interesting insight is the trace's values, namely the values drawn during sampling that reflect the target posterior distribution.

For each variable to be estimated in our probabilistic model, the `traceplot` function provides two kinds of plots:

* a *frequency plot* (left) that shows the output of a kernel density estimation (KDE) operation on the drawn samples for each chain. You should check that all chains follow the same trend and that the estimated density is unimodal. If the latter is not verified, you may need to run more draws.

* a series of values taken by every chain at each sampling step (right): you should check that all chains explore efficiently the search space (absence of stale sequences where the chain keeps the same value for a while)

It is also a good practice to discard the first samples that are not representative of the target distribution we aim at estimating (a.k.a. *burn-in* samples).

In [ ]:
n_burnin = 1000

pm.plots.traceplot(trace=trace,
                   skip_first=n_burnin,
                   varnames=["mu_lk"],
                   figsize=(10, 4));

### Posterior density histogram

This detailed illustration of the drawn samples' distribution provides a good visual overview of the posterior distribution(s) as well as some statistical indicators.

In [ ]:
pm.plots.plot_posterior(trace=trace["mu_lk"],
                        figsize=(10, 4))

### Matching observations against inferred distribution

Recall that our main goal is to provide a probabilistic estimation of the parameters that define the generative process from which the observations should be coming from. 

To verify the quality of our inference, we can use the average of the samples to produce a density estimation that we can overlay with the actual observations:

In [ ]:
plt.hist(obs, bins=32, histtype="step", normed=True, color="k", lw=2, label="Observations");

n_pts_kde = 200
color_kde = "#A60628"
if lk_type=="normal":
    x = np.linspace(np.min(obs), np.max(obs), n_pts_kde)
    y = scipy.stats.norm.pdf(x, loc=trace["mu_lk"].mean(axis=0), scale=prior_sigma)
if lk_type=="poisson":
    x = np.arange(1, int(np.floor(np.max(obs))))
    y = scipy.stats.poisson.pmf(x, mu=np.floor(trace["mu_lk"].mean(axis=0)))
plt.plot(x, y, label="Inferred distribution", lw=3, color=color_kde)
plt.fill_between(x, y, color=color_kde, alpha=0.3)
plt.legend(loc="upper right")
plt.title("Observation histogram vs inferred distribution (averaging over samples)")
plt.xlabel(obs_column);